In [2]:
text = """Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."""

In [3]:
tokens = text.encode("utf-8")
tokens= list(map(int, tokens))

print(text)
print(len(text))


print(tokens)
print(len(tokens))

Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.
533
[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226, 128, 140, 240, 159, 135, 180, 226, 128, 140, 240, 159, 135,

In [26]:
def find_pairs(tokens):
    pairs = {}
    for idx, token in enumerate(tokens[:-1]): # better to use zip(tokens, tokens[1:])
        pair = (token, tokens[idx+1])
        pairs[pair] = pairs.get(pair, 0) + 1
    return pairs

In [27]:
def find_max_occuring_pair(tokens):
    pairs = find_pairs(tokens)
    max_pairs = max(pairs, key=pairs.get) 
    return max_pairs if pairs[max_pairs] > 1 else None # if no pairs occurs return None
    # return pairs

In [5]:
def replace_pair(tokens:list, pair:tuple, new_token:int) -> list:
    tokens_copy = tokens.copy()
    remove_index = []
    for idx, t_pair in enumerate(zip(tokens_copy, tokens[1:])):
        if pair == t_pair:
            tokens_copy[idx] = new_token
            remove_index.append(idx+1)

    # remove index
    for idx in sorted(remove_index, reverse=True):
        del tokens_copy[idx]
    return tokens_copy

In [6]:
def bytepair_encoding(text:str, vocab_size:int):
    tokens = text.encode("utf-8")
    tokens = list(map(int, tokens))
    new_token = 256
    merge_dict = {}
    while new_token <= vocab_size:
        token_pair = find_max_occuring_pair(tokens)
        if not token_pair: # if no pair occurs
            print("No more pairs to merge")
            break
        merge_dict[token_pair] = new_token
        print(f"Token Pair: {token_pair}")
        tokens= replace_pair(tokens, token_pair, new_token)
        new_token+= 1
    return tokens, merge_dict

In [7]:
encoded_tokens, merge_dict = bytepair_encoding(text, 276)

Token Pair: (101, 32)
Token Pair: (240, 159)
Token Pair: (226, 128)
Token Pair: (105, 110)
Token Pair: (115, 32)
Token Pair: (97, 110)
Token Pair: (116, 104)
Token Pair: (257, 133)
Token Pair: (257, 135)
Token Pair: (97, 114)
Token Pair: (239, 189)
Token Pair: (258, 140)
Token Pair: (267, 264)
Token Pair: (101, 114)
Token Pair: (111, 114)
Token Pair: (116, 32)
Token Pair: (259, 103)
Token Pair: (115, 116)
Token Pair: (261, 100)
Token Pair: (32, 262)
Token Pair: (44, 32)


In [8]:
merge_dict

{(101, 32): 256,
 (240, 159): 257,
 (226, 128): 258,
 (105, 110): 259,
 (115, 32): 260,
 (97, 110): 261,
 (116, 104): 262,
 (257, 133): 263,
 (257, 135): 264,
 (97, 114): 265,
 (239, 189): 266,
 (258, 140): 267,
 (267, 264): 268,
 (101, 114): 269,
 (111, 114): 270,
 (116, 32): 271,
 (259, 103): 272,
 (115, 116): 273,
 (261, 100): 274,
 (32, 262): 275,
 (44, 32): 276}

In [41]:
def encode(text:str):
    tokens = text.encode("utf-8")
    tokens= list(map(int, tokens))
    while True and len(tokens) > 1:
        # get merge_pair
        merge_pairs = find_pairs(tokens)
        merge_pair = min(merge_pairs, key=lambda x: merge_dict.get(x, float("inf")))
        # if no merge pair found
        if merge_dict.get(merge_pair) is None:
            print("No more merge pairs")
            break
        # get new token
        new_token = merge_dict[merge_pair]
        # replace pair
        tokens = replace_pair(tokens, merge_pair, new_token)
    return tokens

In [42]:
encode("hello world")

No more merge pairs


[104, 101, 108, 108, 111, 32, 119, 270, 108, 100]

In [11]:
vocab = {idx: bytes([idx]) for idx in range(256)}

In [12]:
for (p0,p1), idx in merge_dict.items():
    vocab[idx] = vocab[p0] + vocab[p1]

In [20]:
def decode(tokens):
    tokens = b"".join(vocab.get(token) for token in tokens)
    text = tokens.decode("utf-8", errors="replace")
    return text

In [43]:
encode("") 

[]

In [39]:
decode([72, 69, 76, 76, 79])

'HELLO'